In [1]:
import os
import gc
import time
import json
import logging
import traceback
from typing import List, Dict, Any, Union, Optional
from typing_extensions import Annotated
import uuid
from datetime import datetime
import logging

from fastapi import (
    Body,
    Depends,
    FastAPI,
    HTTPException,
    Request,
    Response,
    status,
    File,
    UploadFile,
)
from fastapi.responses import HTMLResponse, JSONResponse, RedirectResponse
from fastapi.exception_handlers import request_validation_exception_handler
from fastapi.exceptions import RequestValidationError
from fastapi.security import OAuth2PasswordBearer

from pydantic import BaseModel, Field, Json, ValidationError
import pandas as pd

import asyncio
#from llm import LLM
import sqlite3
df_file = "db_project.db"

app = FastAPI()
#llm = LLM()

In [2]:
table_name = "Task"

conn = sqlite3.connect(df_file)
cursor = conn.cursor()
cursor.execute(f"PRAGMA table_info({table_name});")
columns = cursor.fetchall()
columns

[(0, 'task_id', 'INTEGER', 1, None, 1),
 (1, 'task_name', 'TEXT', 0, None, 0),
 (2, 'task_category', 'TEXT', 0, None, 0),
 (3, 'task_description', 'TEXT', 0, None, 0)]

In [3]:
sql_insert = '''INSERT INTO Task (task_name, task_category, task_description)
                         VALUES (?, ?, ?)'''

In [4]:
task_name = "Автопутешествие"
task_category = "Математика"
task_description = '''Представьте, что вы отправились в автопутешествие с напарником. Длина маршрута - 2300 километров. Между пунктами назначения вы движетесь со средней скоростью 80 км/ч. Дневные остановки занимают от двух до восьми часов. Также ночью вы обязательно ложитесь спать, чтобы не подвергать себя риску попасть в аварию. Вы находитесь в пути уже два дня, и вам любопытно, сколько процентов маршрута вы успели преодолеть. А ваш напарник интересуется, сколько всего дней продлится это путешествие. Посчитайте вероятность того, что ваш ответ на вопрос напарника окажется верным. Получите ответы на эти три вопроса с помощью правильного запроса к языковой модели.'''

In [7]:
task_name = "Старинная деревня"
task_category = "Лирическая задача"
task_description = """Вообразите, что вы переместились в далёкое прошлое параллельной вселенной. Вам повезло, и вы очутились в старинной деревне, в которой знают ваш язык. Жители признали в вас незнакомца, но вы не хотите выделяться из толпы во избежание неприятностей. Вы представились путешественником по имени Хонго, а в качестве доказательства решили рассказать о самой солнечной погоде, которую вам довелось застать некотором государстве Миуо. Чтобы вашу ложь не раскрыли, ваша речь должна содержать лишь самую простую и понятную лексику, присущую разговорному стилю. Притом рассказ должен быть детальным и интересным, дабы жители прониклись вашей историей и забыли о своих подозрениях. Получите это описание с помощью правильного запроса к языковой модели."""

In [9]:
task_name = 'Логические сладости'
task_category = 'Головоломка'
task_description = '''Мальчику Кё всего пять лет, и он без ума от шоколадных пирожных и марципана. Его старшая сестра Който уже учится в средней школе и увлекается математикой. Однажды девочка возвращается домой с учёбы и угощает младшего брата одну конфетой. На следующий день Който угостила мальчика ещё одной конфетой. Кё очень радовался, но даже не догадывался о поводе для угощений. В последующие дни Който угостила брала двумя, тремя, а затем и пятью конфетами. Мальчик был счастлив, но начал подозревать сестру в чём-то неладном. На следующий день сестра принесла уже 8 конфет. Для чего же столько сладостей? По какому принципу Който выбирает число конфет? Сколько сладостей девочка принесёт на следующий день? Получите ответ на этот вопрос с помощью правильного запроса к языковой модели.'''

In [10]:
conn = sqlite3.connect(df_file)
cursor = conn.cursor()
cursor.execute(sql_insert, (task_name, task_category, task_description))
conn.commit() # Save changes
cursor.execute("SELECT task_description FROM Task")
tasks = cursor.fetchall()
conn.close()
tasks

[('Представьте, что вы отправились в автопутешествие с напарником. Длина маршрута - 2300 километров. Между пунктами назначения вы движетесь со средней скоростью 80 км/ч. Дневные остановки занимают от двух до восьми часов. Также ночью вы обязательно ложитесь спать, чтобы не подвергать себя риску попасть в аварию. Вы находитесь в пути уже два дня, и вам любопытно, сколько процентов маршрута вы успели преодолеть. А ваш напарник интересуется, сколько всего дней продлится это путешествие. Посчитайте вероятность того, что ваш ответ на вопрос напарника окажется верным. Получите ответы на эти три вопроса с помощью правильного запроса к языковой модели.',),
 ('Вообразите, что вы переместились в далёкое прошлое параллельной вселенной. Вам повезло, и вы очутились в старинной деревне, в которой знают ваш язык. Жители признали в вас незнакомца, но вы не хотите выделяться из толпы во избежание неприятностей. Вы представились путешественником по имени Хонго, а в качестве доказательства решили рассказ

In [202]:
task_id=2

conn = sqlite3.connect(df_file)
cursor = conn.cursor()
cursor.execute(f"SELECT * FROM Task WHERE task_id = {task_id}")
task = cursor.fetchone()
conn.close()
task

(2,
 'Старинная деревня',
 'Лирическая задача',
 'Вообразите, что вы переместились в далёкое прошлое параллельной вселенной. Вам повезло, и вы очутились в старинной деревне, в которой знают ваш язык. Жители признали в вас незнакомца, но вы не хотите выделяться из толпы во избежание неприятностей. Вы представились путешественником по имени Хонго, а в качестве доказательства решили рассказать о самой солнечной погоде, которую вам довелось застать некотором государстве Миуо. Чтобы вашу ложь не раскрыли, ваша речь должна содержать лишь самую простую и понятную лексику, присущую разговорному стилю. Притом рассказ должен быть детальным и интересным, дабы жители прониклись вашей историей и забыли о своих подозрениях. Получите это описание с помощью правильного запроса к языковой модели.')

In [183]:
table_name = 'Task' 
conn = sqlite3.connect(df_file)
cursor = conn.cursor()
sql_delete = f'DELETE FROM {table_name}'
cursor.execute(sql_delete)
# Delete from sqlite_sequence to reset auto-increment
cursor.execute("DELETE FROM sqlite_sequence WHERE name = 'Task';")
conn.commit()
conn.close()